<a href="https://colab.research.google.com/github/charlie-may86/twitoff2/blob/master/DSPT6_APIs_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install basilica

In [2]:
import tweepy
import basilica

In [4]:
## ADD YOUR CREDENTIALS HERE
# TWITTER_CONSUMER_API_KEY = 
# TWITTER_CONSUMER_API_SECRET = 
# TWITTER_ACCESS_TOKEN = 
# TWITTER_ACCESS_TOKEN_SECRET = 
BASILICA_KEY = 'b73e93fb-9406-1a2e-1b5a-a67763b69679'

In [17]:
# TWITTER_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY,TWITTER_CONSUMER_API_SECRET)
# TWITTER_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
# TWITTER = tweepy.API(TWITTER_AUTH)
BASILICA = basilica.Connection(BASILICA_KEY)

In [ ]:
#@title
dir(TWITTER)

In [ ]:
#@title
twitter_user = TWITTER.get_user('elonmusk')
twitter_user

In [ ]:
#@title
twitter_user.id

In [ ]:
#@title
tweets = twitter_user.timeline()
tweets

In [ ]:
#@title
tweets[1].text

In [ ]:
#@title
tweets = twitter_user.timeline(count=200,
                               exclude_replies=True,
                               include_rts=False,
                               # since=1,
                               tweet_mode='extended')
print(len(tweets))

In [ ]:
#@title
tweets[1].full_text

In [ ]:
#@title
embedding = BASILICA.embed_sentence(tweets[1].full_text, model='twitter')
len(embedding)

In [ ]:
#@title
def add_user_tweepy(username):
  '''Add a user and their tweets to database'''
  try:
    # Get user info from tweepy
    twitter_user = TWITTER.get_user(username)

    # Get tweets ignoring re-tweets and replies
    tweets = twitter_user.timeline(count=200,
                                   exclude_replies=True,
                                   include_rts=False,
                                   tweet_mode='extended')
    
    # Get an examble basilica embedding for first tweet
    embedding = BASILICA.embed_sentence(tweets[0].full_text, model='twitter')

  except Exception as e:
    print('Error processing {}: {}'.format(username, e))
    raise e

  return tweets, embedding

In [ ]:
#@title
elon_tweets, elon_embedding = add_user_tweepy('elonmusk')
print(len(elon_tweets), len(elon_embedding))

### Twitter Scraper (for those without API access)

In [ ]:
!pip install twitter_scraper

In [6]:
from twitter_scraper import get_tweets, Profile

In [7]:
elon_profile = Profile('elonmusk')
dir(elon_profile)

['banner_photolikes_count',
 'biography',
 'birthday',
 'followers_count',
 'following_count',
 'is_private',
 'is_verified',
 'location',
 'name',
 'profile_photo',
 'tweets_count',
 'user_id',
 'username',
 'website']

In [8]:
elon_profile.tweets_count

11948

In [9]:
tweets = list(get_tweets('elonmusk', pages=25))
print(len(tweets))

499


In [11]:
tweets[0:3]

[{'entries': {'hashtags': ['#Tesla', '#GigaBerlin'],
   'photos': [],
   'urls': ['https://twitter.com/torquenewsauto/status/1291372623402881024'],
   'videos': []},
  'isPinned': False,
  'isRetweet': False,
  'likes': 19428,
  'replies': 1106,
  'retweets': 1092,
  'text': 'Good piece on Giga Berlin construction https://twitter.com/torquenewsauto/status/1291372623402881024\xa0…',
  'time': datetime.datetime(2020, 8, 7, 17, 12, 8),
  'tweetId': '1291784200178159616',
  'tweetUrl': '/elonmusk/status/1291784200178159616',
  'userId': '44196397',
  'username': 'elonmusk'},
 {'entries': {'hashtags': [],
   'photos': [],
   'urls': [],
   'videos': [{'id': 've_2UFK8GFTnHOZ4'}]},
  'isPinned': False,
  'isRetweet': True,
  'likes': 42496,
  'replies': 692,
  'retweets': 5041,
  'text': 'Falcon 9’s first stage has landed on the Of Course I Still Love You droneship pic.twitter.com/szO3thMxqa',
  'time': datetime.datetime(2020, 8, 7, 5, 21, 49),
  'tweetId': '1291605444809523200',
  'tweetUrl'

In [12]:
original_tweets = [i for i in tweets if i['username']=='elonmusk']
print(len(original_tweets))

363


In [13]:
original_tweets[0]['text']

'Good piece on Giga Berlin construction https://twitter.com/torquenewsauto/status/1291372623402881024\xa0…'

In [15]:
def add_user_twitter_scraper(username):
    """Add a user and their tweets to database."""
    try:
        # Get user profile   
        user_profile = list(get_tweets(username, pages=25))

        # Get most recent tweets
        tweets = list(get_tweets(username, pages=10))
        original_tweets = [i for i in tweets if i['username']==username]

        # Get an example Basilica embedding for first tweet
        embedding = BASILICA.embed_sentence(original_tweets[0]['text'], model='twitter')
            
    except Exception as e:
        print('Error processing {}: {}'.format(username, e))
        raise e

    return original_tweets, embedding

In [18]:
elon_tweets, elon_embedding = add_user_twitter_scraper('elonmusk')
print(len(elon_tweets), len(elon_embedding))

132 768
